### Import Libraries

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.cm as cm
import plotly.express as px
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.metrics import accuracy_score
from keras.models import Model
from keras import layers, losses


from keras.models import Sequential
from keras.layers import InputLayer, LSTM, Dense, RepeatVector, TimeDistributed
from keras.utils import plot_model

In [2]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type != 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

### Load data

In [3]:
DATA_DIR_NORMAL = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/data/anomalyDetection_Normal.npy"
DATA_DIR_ISSUE = "/Users/kedi/Desktop/polito lectures and notes/ML in App/project/data/anomalyDetection_Issue.npy"

# Set style for matplotlib
plt.style.use("Solarize_Light2")

In [4]:
data_normal = np.load(DATA_DIR_NORMAL)
data_issue = np.load(DATA_DIR_ISSUE)

data_normal = data_normal.reshape((data_normal.shape[0], data_normal.shape[1], 1))
data_issue = data_issue.reshape((data_issue.shape[0], data_issue.shape[1], 1))

## Model

In [14]:
input_shape = (data_normal.shape[1:])
# define model
model = Sequential()
model.add(LSTM(85, activation='relu', input_shape=input_shape, kernel_initializer="random_normal", bias_initializer='zeros'))
model.add(RepeatVector(input_shape[0]))
model.add(LSTM(85, activation='relu', return_sequences=True, kernel_initializer="random_normal", bias_initializer='zeros'))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_2 (LSTM)               (None, 85)                29580     
                                                                 
 repeat_vector_1 (RepeatVect  (None, 85, 85)           0         
 or)                                                             
                                                                 
 lstm_3 (LSTM)               (None, 85, 85)            58140     
                                                                 
 time_distributed_1 (TimeDis  (None, 85, 1)            86        
 tributed)                                                       
                                                                 
Total params: 87,806
Trainable params: 87,806
Non-trainable params: 0
_________________________________________________________________


In [13]:
try:
    # Disable all GPUS
    tf.config.set_visible_devices([], 'GPU')
    visible_devices = tf.config.get_visible_devices()
    for device in visible_devices:
        assert device.device_type == 'GPU'
except:
    # Invalid device or cannot modify virtual devices once initialized.
    pass

In [18]:
# should not be trained
with tf.device('/GPU'):
    print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
    model.fit(data_normal, data_normal, epochs=100)

Num GPUs Available:  1
Epoch 1/100
32/32 [==============================] - 2s 63ms/step - loss: 0.2080
Epoch 2/100
32/32 [==============================] - 2s 62ms/step - loss: 0.2029
Epoch 3/100
32/32 [==============================] - 2s 62ms/step - loss: 0.2001
Epoch 4/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1945
Epoch 5/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1954
Epoch 6/100
32/32 [==============================] - 2s 62ms/step - loss: 0.1958
Epoch 7/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1883
Epoch 8/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1874
Epoch 9/100
32/32 [==============================] - 2s 62ms/step - loss: 0.1785
Epoch 10/100
32/32 [==============================] - 2s 66ms/step - loss: 0.1824
Epoch 11/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1729
Epoch 12/100
32/32 [==============================] - 2s 63ms/step - loss: 0.1905
Ep

In [19]:
y_train = model.predict(data_normal)

cost_train = y_train - data_normal
print(np.round(cost_train,3), f"mean: {np.round(cost_train.mean(),3)}")

32/32 [==============================] - 1s 18ms/step
[[[ 0.668]
  [ 0.275]
  [-0.138]
  ...
  [ 0.61 ]
  [-0.586]
  [ 0.607]]

 [[ 0.668]
  [ 0.275]
  [-0.138]
  ...
  [ 0.61 ]
  [-0.586]
  [ 0.607]]

 [[ 0.668]
  [ 0.275]
  [-0.138]
  ...
  [ 0.61 ]
  [-0.586]
  [ 0.607]]

 ...

 [[ 0.416]
  [ 0.482]
  [ 0.366]
  ...
  [ 0.698]
  [-0.17 ]
  [ 0.599]]

 [[ 0.417]
  [ 0.483]
  [ 0.366]
  ...
  [ 0.707]
  [-0.096]
  [ 0.603]]

 [[ 0.417]
  [ 0.483]
  [ 0.366]
  ...
  [ 0.711]
  [-0.064]
  [ 0.604]]] mean: 0.004999999888241291


In [20]:
y_test = model.predict(data_issue)

cost_test = y_test - data_issue
print(np.round(cost_test,3), f"mean: {cost_test.mean()}")

13/13 [==============================] - 0s 19ms/step
[[[-0.374]
  [-0.232]
  [ 0.375]
  ...
  [-1.075]
  [-1.066]
  [ 0.925]]

 [[-0.374]
  [-0.232]
  [ 0.375]
  ...
  [-1.077]
  [-1.065]
  [ 0.925]]

 [[ 0.05 ]
  [-0.042]
  [ 0.375]
  ...
  [-1.077]
  [-1.065]
  [ 0.925]]

 ...

 [[ 0.371]
  [ 0.205]
  [ 0.375]
  ...
  [-0.79 ]
  [-1.286]
  [ 0.921]]

 [[ 0.371]
  [ 0.205]
  [ 0.375]
  ...
  [-0.747]
  [-1.306]
  [ 0.921]]

 [[ 0.371]
  [ 0.205]
  [ 0.375]
  ...
  [-0.71 ]
  [-1.32 ]
  [ 0.921]]] mean: 0.02613138221204281
